## Auto Correct

## Data Processing

In [1]:
/ Load the shakespeare.txt file from Data folder

/Note our corpus going to be based on this text file only. 
/This model won't suggest the word, apart from this text file

word_l: read0 `:./../data/shakespeare.txt

alpha_chars : ((.Q.a),(.Q.n),("_"));
punctuations:"!@#$%^&*()?:;,.`~<>+\"={|}/'";
word_l:(" " sv word_l)

/preprocessing. Extract the each word from the text file and build frequency table.
word_l:ssr/[word_l;punctuations;" "]
word_l: " " vs word_l
word_l: `$(lower(word_l except enlist""))

In [2]:
/Get count function

word_count_dict:count each group word_l
probs: word_count_dict % (count word_count_dict)

In [276]:
count probs

6314


In [277]:
/We have 6.3k words in out corpus. If the user enter any of these misspelled words, it will suggest the correct
/ or related word

probs_table:([word:key probs] prob:(value probs))

In [241]:
probs_table

word     | prob        
---------| ------------
o        | 0.02486538  
for      | 0.07491289  
a        | 0.118942    
muse     | 0.002850808 
of       | 0.172949    
fire     | 0.003484321 
that     | 0.1240101   
would    | 0.02169781  
ascend   | 0.0001583782
the      | 0.2408933   
brightest| 0.0003167564
heaven   | 0.007602154 
invention| 0.001425404 
kingdom  | 0.0009502692
stage    | 0.0004751346
princes  | 0.0006335128
to       | 0.1837187   
act      | 0.005543237 
and      | 0.2207792   
monarchs | 0.0003167564
..


<a name='2'></a>
## String Manipulations

Compute the $P(w_i)$ for all the words in the corpus, have to write a few functions to manipulate strings so that we can edit the erroneous strings and return the right spellings of the words.

* `delete_letter`: given a word, it returns all the possible strings that have **one character removed**. 
* `switch_letter`: given a word, it returns all the possible strings that have **two adjacent letters switched**.
* `replace_letter`: given a word, it returns all the possible strings that have **one character replaced by another different letter**.
* `insert_letter`: given a word, it returns all the possible strings that have an **additional character inserted**. 


<a name='ex-4'></a>
### delete_letter

**Idelete_letter():** Implement a `delete_letter()` function that, given a word, returns a list of strings with one character deleted. 

For example, given the word **nice**, it would return the set: {'ice', 'nce', 'nic', 'nie'}. 

In [158]:
delete_letter: {$[(count x) > 1;(x except/: (x));""]}

In [150]:
delete_letter "cans"

"ans"
"cns"
"cas"
"can"


In [151]:
delete_letter "at"

,"t"
,"a"


<a name='ex-5'></a>
### switch_letter

**switch_letter()**: a function that switches two letters in a word. It takes in a word and returns a list of all the possible switches of two letters **that are adjacent to each other**. 
- For example, given the word 'eta', it returns {'eat', 'tea'}, but does not return 'ate'.

In [6]:
switch_letter: { s::x;{@[s;x;:;s[reverse x]]}'[({(x+til 2)}'[til ((count s)-1)])]}

In [7]:
switch_letter "eta"

"tea"
"eat"


In [25]:
switch_letter "senthil"

"esnthil"
"snethil"
"setnhil"
"senhtil"
"sentihl"
"senthli"


<a name='ex-6'></a>
### replace_letter
**replace_letter()**: a function that takes in a word and returns a list of strings with one **replaced letter** from the original word. 

In [15]:
replace:{s::x;{{@[s;x;:;y]}'[x;.Q.a]}'[til (count s)]}

In [155]:
replace_letter: {$[(count x)>1;((raze (replace x)) except enlist x);x]}

In [156]:
replace_letter "can"

"aan"
"ban"
"dan"
"ean"
"fan"
"gan"
"han"
"ian"
"jan"
"kan"
"lan"
"man"
"nan"
"oan"
"pan"
"qan"
"ran"
"san"
"tan"
"uan"
"van"
"wan"
..


In [153]:
count replace_letter "at"

50


<a name='ex-7'></a>
###  Insert_letter

**Instructions for insert_letter()**: a function that takes in a word and returns a list with a letter inserted at every offset.


In [278]:
/You may get confusion on insert_letter function

In [131]:
/This function will give the split index. Between these index the new letter will be added.
cutt: {[x;y] ((til x);(x _ (til y)))}

In [132]:
/Split the word into two part based on cutt function index.
split:{s::x;{s@cutt[x;count s]}'[til (count s)+1]}

In [279]:
/ This function will handle the single word like "a","i","u".
single_letter_handling:{enlist (((cross/) (x;.Q.a)) union ((cross/) (.Q.a;x)))}

In [281]:
/ In this inserting function, the new word will be going to add between the splited words.
/Ex: can --> "c","an" --> "c","a"[new_letter],"an" -> "caan" 
/can --> "c","an" --> "c","l"[new_letter],"an"  --> "clan"
/ may --> "m","ay" --> "m","r","ay" -> "mray"
/..
/..
/ may --> "ma","y" --> "ma","n","y" --> "many"
inserting: {$[(count x) > 1;{[wrd] ((cross/) (enlist wrd[0];.Q.a;enlist wrd[1]))}'[split x];single_letter_handling x]}

insert_letter:{raze (inserting x)}

In [185]:
insert_letter "at"

"aat"
"bat"
"cat"
"dat"
"eat"
"fat"
"gat"
"hat"
"iat"
"jat"
"kat"
"lat"
"mat"
"nat"
"oat"
"pat"
"qat"
"rat"
"sat"
"tat"
"uat"
"vat"
..


In [186]:
count insert_letter "a"

51


<a name='3'></a>
## Combining the Edits

Then create two functions that, given a string, will return all the possible single and double edits on that string. These will be `edit_one_letter()` and `edit_two_letters()`.

<a name='3-1'></a>
### Edit One Letter

<a name='ex-8'></a>
### edit_one_letter

the `edit_one_letter` function to get all the possible edits that are one edit away from a word. The edits  consist of the replace, insert, delete, and optionally the switch operation. The 'switch' function  is a less common edit function, so its use will be selected by an "allow_switches" input argument.


In [296]:
/Combine the all possible ways like switch, replace, delete, insert

edit_one_letter:{[x;f]
    switch_let:$[f;switch_letter[x];()];
    rep_let: replace_letter[x];
    del_let:delete_letter[x];
    ins_let:insert_letter[x];
    :(switch_let union del_let union del_let union ins_let)
    }

In [148]:
count edit_one_letter ["can";1b]

106


In [282]:
count edit_one_letter ["a";1b]

51


<a name='3-2'></a>
###  Edit Two Letters

<a name='ex-9'></a>
### edit_two_letters

the `edit_two_letters` function that returns a set of words that are two edits away. Note that creating additional edits based on the `edit_one_letter` function may 'restore' some one_edits to zero or one edits. That is allowed here. This is accounted for in get_corrections.

In [284]:
edit_two_letters:{[x;f]
    ittr1:distinct(edit_one_letter[x;f]);
    ittr2:edit_one_letter'[ittr1;]f;
    :(ittr1 union (asc distinct (raze ittr2)) )except enlist""
    }

In [285]:
edit_two_letters["a";1b]

"aa"
"ab"
"ac"
"ad"
"ae"
"af"
"ag"
"ah"
"ai"
"aj"
"ak"
"al"
"am"
"an"
"ao"
"ap"
"aq"
"ar"
"as"
"at"
"au"
"av"
..


In [283]:
count edit_two_letters["a";1b]

2028


<a name='3-3'></a>
### Suggest Spelling Suggestions

Now we can use `edit_two_letters` function to get a set of all the possible 2 edits on our corpus. then use those strings to get the most probable word user meant to type a.k.a typing suggestion.

<a name='ex-10'></a>
### get_corrections
The `get_corrections`, which returns a list of zero to n possible suggestion table form (word, probability_of_word). The top word will have high probability.

Generate suggestions for a supplied word: The 'suggestion algorithm' should follow this logic: 
* If the word is in the vocabulary, suggest the word. 
* Otherwise, if there are suggestions from `edit_one_letter` that are in the vocabulary, use those. 
* Otherwise, if there are suggestions from `edit_two_letters` that are in the vocabulary, use those. 
* Otherwise, suggest the input word.*  
* The idea is that words generated from fewer edits are more likely than words with more edits.



In [231]:
probs

o        | 0.02486538
for      | 0.07491289
a        | 0.118942
muse     | 0.002850808
of       | 0.172949
fire     | 0.003484321
that     | 0.1240101
would    | 0.02169781
ascend   | 0.0001583782
the      | 0.2408933
brightest| 0.0003167564
heaven   | 0.007602154
invention| 0.001425404
kingdom  | 0.0009502692
stage    | 0.0004751346
princes  | 0.0006335128
to       | 0.1837187
act      | 0.005543237
and      | 0.2207792
monarchs | 0.0003167564
behold   | 0.002058917
swelling | 0.0001583782
..


In [417]:
get_corrections:{
    words_list: `$(edit_two_letters[x;1b]);
    :(`prob xdesc select from probs_table where word in words_list)
    };


get_corrections_1:{
    words_list: `$(edit_two_letters[x;1b]);
    res:string ((exec word from (`prob xdesc select from probs_table where word in words_list))[0]);
    /$[`$res = `;x;res];
    :res
    };



In [347]:
get_corrections "smpl"

word  | prob        
------| ------------
simple| 0.001108647 
simply| 0.0004751346


In [348]:
/Usualy in our mobile phone, they suggest top three words
get_corrections "img"

word | prob        
-----| ------------
i    | 0.2001901   
him  | 0.04339563  
image| 0.0006335128
aim  | 0.0004751346
big  | 0.0003167564
m    | 0.0001583782


In [350]:
get_corrections "trvl"

word  | prob       
------| -----------
travel| 0.001108647


In [419]:
/Let's add on sentence
sentenct_auto_correct:{
    x: " " vs x;
    x: (get_corrections_1'[x]);
    x:(" " sv x);
    :x
    }

In [420]:
sentenct_auto_correct "welcme evry one"

"welcome every no"
